In [44]:
import pyspark.sql.functions as f
from pyspark.sql.types import *

import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.float_format', '{:0.3f}'.format)

from pprint import pprint

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
# Load training data
df = spark.sql("select * from training.training_data")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
# check NA
df.select([f.count(f.when(f.isnan(c[0]) | f.isnull(c[0]), c[0])).alias(c[0]) for c in df.dtypes if not c[1] in ("string", "timestamp", "array<string>")])\
.toPandas().transpose().sum().sum() == 0

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

True

In [4]:
# Potential target variables
df.agg(f.mean(("answer_accepted_flag"))).show()
df.agg(f.mean(("answer_accepted_1d_flag"))).show()
df.agg(f.mean(("answer_accepted_7d_flag"))).show()
df.agg(f.mean(("answer_accepted_14d_flag"))).show()
df.agg(f.mean(("answer_accepted_30d_flag"))).show()

df.agg(f.mean(("answer_flag"))).show()
df.agg(f.mean(("answer_1d_flag"))).show()
df.agg(f.mean(("answer_7d_flag"))).show()
df.agg(f.mean(("answer_14d_flag"))).show()
df.agg(f.mean(("answer_30d_flag"))).show()

df.agg(f.mean(("post_closed_flag"))).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------------+
|avg(answer_accepted_flag)|
+-------------------------+
|      0.37565655757954775|
+-------------------------+

+----------------------------+
|avg(answer_accepted_1d_flag)|
+----------------------------+
|         0.14489200888418927|
+----------------------------+

+----------------------------+
|avg(answer_accepted_7d_flag)|
+----------------------------+
|         0.28837452992207835|
+----------------------------+

+-----------------------------+
|avg(answer_accepted_14d_flag)|
+-----------------------------+
|           0.3083902121153894|
+-----------------------------+

+-----------------------------+
|avg(answer_accepted_30d_flag)|
+-----------------------------+
|           0.3227118498424023|
+-----------------------------+

+------------------+
|  avg(answer_flag)|
+------------------+
|0.7951182310867856|
+------------------+

+-------------------+
|avg(answer_1d_flag)|
+-------------------+
| 0.5677371693044339|
+-------------------+

+-------

In [13]:
df.groupBy("dataset_name").agg(f.mean("answer_accepted_flag")).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------------------------+
|        dataset_name|avg(answer_accepted_flag)|
+--------------------+-------------------------+
|cs-stackexchange-com|       0.4482627578718784|
|datascience-stack...|       0.3252392907578848|
|softwareengineeri...|       0.5755453094520305|
|android-stackexch...|      0.24668830241054845|
|unix-stackexchang...|      0.44735514678683125|
|dba-stackexchange...|      0.46438227050623804|
|devops-stackexcha...|       0.3636882671995432|
|gamedev-stackexch...|       0.4996215479769036|
|       askubuntu-com|       0.2853066188245777|
|raspberrypi-stack...|       0.3082882018235415|
+--------------------+-------------------------+

In [14]:
df.groupBy("dataset_name").agg(f.mean("answer_flag")).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------------+
|        dataset_name|  avg(answer_flag)|
+--------------------+------------------+
|cs-stackexchange-com|0.7833876221498371|
|datascience-stack...|0.7485093362623568|
|softwareengineeri...|0.9534846604007803|
|android-stackexch...|0.7246341314065274|
|unix-stackexchang...|0.8246236450661913|
|dba-stackexchange...|0.8665947367115608|
|devops-stackexcha...|0.8321438766771339|
|gamedev-stackexch...|0.8586535758309725|
|       askubuntu-com| 0.742233687734043|
|raspberrypi-stack...|0.7819633920614245|
+--------------------+------------------+

In [38]:
# Features vs target
y = "answer_accepted_7d_flag"
x = [
"post_hour",
"post_dayofweek",
"post_month",    
"post_year",
"post_body_char_count",
"post_body_nocode_char_count",
"post_body_code_perc",
"post_body_code_flag",
"post_body_image_flag",
"post_body_link_flag",
"post_body_bold_flag",
"post_title_upper_flag",
"post_title_question_flag",
"post_title_char_count",
"post_tag_count",
"post_body_sent_count",
"post_body_word_count",
"post_title_word_count",
"tag_post_count_max",
"tag_post_count_30d_max",
"tag_post_count_365d_max",
"tag_post_count_avg",
"tag_post_count_30d_avg",
"tag_post_count_365d_avg",
"user_age_days",
"user_age_months",
"user_website_flag",
"user_location_flag",
"user_about_me_flag",
"user_badge_count",
"user_badge_1_count",
"user_badge_2_count",
"user_badge_3_count",
"user_post_count",
"user_question_count",
"user_answer_count",
"user_first_post_flag",
"user_first_question_flag",
"user_answered_questions_count",
"user_accepted_answers_count",
"user_score",
"user_question_score",
"user_answer_score"
]

df_model = df.select(*x, y)

x_mean_vs_y = \
df_model.groupBy(y)\
.agg(*[f.mean(c[0]).alias(c[0]) for c in df_model.dtypes if not c[1] in ("string", "timestamp", "array<string>")] )\
.toPandas().transpose()

x_median_vs_y = \
df_model.groupBy(y)\
.agg(*[f.expr(f"percentile_approx({c[0]}, 0.5)").alias(c[0]) for c in df_model.dtypes if not c[1] in ("string", "timestamp", "array<string>")] )\
.toPandas().transpose()

x_vs_y = x_mean_vs_y.merge(x_median_vs_y, 
                           left_index=True, 
                           right_index=True,
                          suffixes=("_mean", "_median"))
x_vs_y

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                                0_mean   1_mean  0_median  1_median
answer_accepted_7d_flag          1.000    0.000     1.000     0.000
answer_accepted_7d_flag          1.000    0.000     1.000     0.000
answer_accepted_7d_flag          1.000    0.000     1.000     0.000
answer_accepted_7d_flag          1.000    0.000     1.000     0.000
post_body_bold_flag              0.145    0.137     0.000     0.000
post_body_char_count           895.109 1022.750   577.000   598.000
post_body_code_flag              0.566    0.476     1.000     0.000
post_body_code_perc              0.322    0.267     0.119     0.000
post_body_image_flag             0.000    0.000     0.000     0.000
post_body_link_flag              0.226    0.249     0.000     0.000
post_body_nocode_char_count    444.993  506.219   322.000   373.000
post_body_sent_count             4.630    5.256     4.000     4.000
post_body_word_count            80.791   91.012    59.000    68.000
post_dayofweek                   3.991    3.996 

In [39]:
train, test = df_model.randomSplit([0.7, 0.3], seed = 1)
df.count(), train.count(), test.count()

train.select(f.mean(y)).show()
test.select(f.mean(y)).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------------------+
|avg(answer_accepted_7d_flag)|
+----------------------------+
|         0.28837863407399833|
+----------------------------+

+----------------------------+
|avg(answer_accepted_7d_flag)|
+----------------------------+
|          0.2883649475837585|
+----------------------------+

In [45]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import GBTClassifier, RandomForestClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer, OneHotEncoderEstimator, VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

from pyspark.mllib.evaluation import MulticlassMetrics

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [71]:
# Baseline model
indexer = StringIndexer(inputCol=y, outputCol='label')
vec_assembler = VectorAssembler(inputCols=x, outputCol="features")
gbt = GBTClassifier()
pipeline = Pipeline(stages=[indexer, vec_assembler, gbt])

model = pipeline.fit(train)
prediction = model.transform(test)

evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
print("Accuracy:", evaluator.evaluate(prediction))

evaluator = BinaryClassificationEvaluator(metricName="areaUnderROC")
print("ROC AUC:", evaluator.evaluate(prediction))

evaluator = BinaryClassificationEvaluator(metricName="areaUnderPR")
print("PR AUC:", evaluator.evaluate(prediction))

metrics = MulticlassMetrics(prediction.select("prediction", "label").rdd.map(tuple))
print("Confusion matrix:\n", metrics.confusionMatrix().toArray()/test.count())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Accuracy: 0.7194636588685733
ROC AUC: 0.6783153959202025
PR AUC: 0.4478980061232942
Confusion matrix:
 [[0.68336133 0.02827373]
 [0.25226261 0.03610233]]

In [72]:
# Model parameters
pprint(model.stages[2].extractParamMap())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{Param(parent='GBTClassifier_cc9334b9098a', name='cacheNodeIds', doc='If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees.'): False,
 Param(parent='GBTClassifier_cc9334b9098a', name='featureSubsetStrategy', doc='The number of features to consider for splits at each tree node. Supported options: auto, all, onethird, sqrt, log2, (0.0-1.0], [1-n].'): 'all',
 Param(parent='GBTClassifier_cc9334b9098a', name='checkpointInterval', doc='set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext'): 10,
 Param(parent='GBTClassifier_cc9334b9098a', name='labelCol', doc='label column name'): 'label',
 Param(parent='GBTClassifier_cc9334b9098a', name='lossType', doc='Loss function which GBT tries to 

In [74]:
# Grid search
paramGrid = ParamGridBuilder() \
.addGrid(gbt.stepSize, [0.05, 0.1, 0.15])\
.build()

crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=BinaryClassificationEvaluator(metricName="areaUnderPR"),
                          numFolds=5)

cvModel = crossval.fit(train)
prediction = cvModel.transform(test)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-74:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 178, in cell_monitor
    job_binned_stages[job_id][stage_id] = all_stages[stage_id]
KeyError: 2675



In [79]:
print("PR AUC:", evaluator.evaluate(prediction))

# Feature importance
pd.DataFrame(model.stages[2].featureImportances.toArray(), index=x)\
.sort_values(0, ascending=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

PR AUC: 0.45362379951954884
                                  0
post_year                     0.110
user_accepted_answers_count   0.103
user_question_count           0.096
post_body_char_count          0.091
tag_post_count_30d_max        0.078
post_body_code_perc           0.064
tag_post_count_365d_max       0.042
post_body_sent_count          0.034
post_title_question_flag      0.031
tag_post_count_max            0.030
tag_post_count_30d_avg        0.030
post_tag_count                0.029
tag_post_count_365d_avg       0.027
user_post_count               0.025
user_age_months               0.023
user_about_me_flag            0.018
post_body_nocode_char_count   0.017
user_badge_count              0.017
post_body_link_flag           0.014
post_hour                     0.014
user_location_flag            0.013
user_answered_questions_count 0.013
user_age_days                 0.012
user_badge_1_count            0.011
user_answer_count             0.011
post_month                    0.008
